In [2]:
%load_ext autoreload
%autoreload 2

from src import *
import tensorflow as tf

In [3]:
batch_size = 1024
epochs = 10
image_feat_path = "./data/test_data/flickr30k/flickr30k_vgg_image_feat_test.mat"
sent_feat_path = "./data/test_data/flickr30k/flickr30k_text_feat_test.mat"
shuffle_step=5

In [4]:
data_loader = DatasetLoader(image_feat_path, sent_feat_path)

Loading image features from ./data/test_data/flickr30k/flickr30k_vgg_image_feat_test.mat
Loaded image feature shape: (1000, 4096)
Loading sentence features from ./data/test_data/flickr30k/flickr30k_text_feat_test.mat
Loaded sentence feature shape: (5000, 6000)


In [51]:
model = embedding_model(data_loader.im_feats.shape, data_loader.sent_feats.shape)
loss = EmbeddingLoss()

In [6]:
img_feats = data_loader.im_feats
sent_feats = data_loader.sent_feats


sent_sample_size = 5 # how many sentences per image

total_loss, batch_loss = [], []
optimizer = tf.keras.optimizers.Adam()

In [48]:
optimizer = tf.keras.optimizers.Adam()

for i in range(1):
    count = i % shuffle_step
    # if count == 0:
    #     data_loader.shuffle_inds()
    im_feats, sent_feats, labels = data_loader.get_batch(count, batch_size, sent_sample_size)
    _batch_loss = tf.constant(0, dtype=tf.float32)
    with tf.GradientTape() as tape:
        # [im_emb, sent_emb] = model(im_feats, sent_feats)
        [im_emb, sent_emb] = model(img_feat_list, sent_feat_list)

        _batch_loss += loss(im_embeds, sent_emb, labels)

    trainable_variables = model.trainable_variables

    gradients = tape.gradient(_batch_loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    # total_loss.append(_batch_loss / int(im_feats.shape[1]))

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,4096], In[1]: [6000,2048] [Op:MatMul]

### currently the data looks like 

| sample size | img_feats | sent_feats |
|-------------|-------------|-------------|
| 1           | img1      | [s1, s2s3, s4, s5] | 
| 2           | img2      | [s1, s2, s3, s4, s5] | 

we need it to be 

|sample size | img_feats | sent_feats|
|-------------|-------------|-------------|
|1           | img1      | s1|
|1           | img1      | s2|
|1           | img1      | s3|
|1           | img1      | s4|
|1           | img1      | s5|
|1           | img2      | s1|
|2           | img2      | s2 and so on

In [8]:
im_feats.shape

(1024, 4096)

In [9]:
sent_feats.shape

(5120, 6000)

In [42]:
feat_list = []

for ind, img in enumerate(im_feats):
    for i in range(0,5):
        feat_list.append([img, sent_feats[ind*5+i]])
    

In [47]:
img_feat_list = []
sent_feat_list = []
for ind, img in enumerate(im_feats):
    for i in range(0,5):
        img_feat_list.append(img)
        sent_feat_list.append(sent_feats[ind*5+i])

In [49]:
print(np.array(img_feat_list).shape)
print(np.array(sent_feat_list).shape)

(5120, 4096)
(5120, 6000)
